### The Notebook contains instructions and code to examine the inconsistencies in the individual station metadata (latitude, longitude, and elevation).

- Some errors in the elevation, latitude, and longitude of individual stations are corrected by replacing these values with EMSHR metadata. (Please see the code below for more information).
- Due to the observational procedure, the metadata of older stations may not be reported precisely.
- In order to account for this, two test are performed:
    - Threshold test
    - One-minute box test for precision

The thresholds used in the threshold and one-minute precision box tests are adjustable by the user.  
The user can <span style="color:red">*modify the values in the cell below the red text*</span>   

The stations that fail the two test are plotted and its information is printed to a csv file.

Inputs:
- Station metadata file (user should provide the path of this file)
- SRTM 90m elevation data (A code is provided on how to generate the data)
- Enhanced Master Station History Report (EMSHR)

Outputs:
- A folium map displaying stations that failed two tests.
- A csv file containing the information of the stations that failed two tests.

Packages:  
Please make sure the following libraries are installed before running the code.  
- Numpy  
- Pandas  
- Xarray  
- Folium  

```Code Author: Sridhar Mantripragada```
```Email: rama.s.mantripragada@noaa.gov```
```Date: 06/21/2023```

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import folium

In [2]:
def read_parse_and_subset_mshr(filename, stnid):
    """
    This function reads the MSHR (Master Station History Report) file, parses it 
    line by line, and subsets it based on provided station ids.

    Parameters:
    filename (str): Name of the MSHR file.
    stnid (list): List of station ids for subsetting the data.

    Returns:
    dfrss (DataFrame): The parsed and subsetted data from the MSHR file.
    """

    def parse_line(line, fmt, column_names):
        """
        This function parses a single line from the MSHR file.

        Parameters:
        line (str): The line of the file to parse.
        fmt (list): The list of length of each data field in the line.
        column_names (list): The list of column names for the data fields.

        Returns:
        row_data (dict): The parsed data in dictionary format.
        """
        row_data = {}  # Initialize an empty dictionary to hold the parsed data
        start = 0  # Initialize starting index of substring
        for i, (col, length) in enumerate(zip(column_names, fmt)):
            end = start + length  # Calculate ending index of substring
            value = line[start:end].strip()  # Extract value and remove leading/trailing whitespaces
            row_data[col] = value  # Store value in dictionary
            start = end

            # Ignore the last space (if any) after the last value
            if i < len(column_names) - 1:  
                start += 1  # Skip the space between values
        
        return row_data
    
    # Define the format and column names for parsing the file
    # fmt defines the length of each data field in a line of the file
    # column_names defines the corresponding name for each data field
    fmt = [20, 10, 8, 8, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 100, 30, 100, 30, 100, 100, 10, 40, 10, 50, 2, 2, 100, 30,
           10, 40, 20, 40, 20, 40, 20, 40, 20, 40, 20, 20, 20, 10, 62, 16, 40, 100, 20, 5, 30, 30, 100, 20, 20]

    column_names = ['SOURCE_ID','SOURCE','BEGIN_DATE','END_DATE','STATION_STATUS','NCDCSTN_ID','ICAO_ID','WBAN_ID','FAA_ID','NWSLI_ID',
                    'WMO_ID','COOP_ID','TRANSMITTAL_ID','GHCND_ID','NAME_PRINCIPAL','NAME_PRINCIPAL_SHORT','NAME_COOP','NAME_COOP_SHORT',
                    'NAME_PUBLICATION','NAME_ALIAS','NWS_CLIM_DIV','NWS_CLIM_DIV_NAME','STATE_PROV','COUNTY','NWS_ST_CODE',
                    'FIPS_COUNTRY_CODE','FIPS_COUNTRY_NAME','NWS_REGION','NWS_WFO','ELEV_GROUND','ELEV_GROUND_UNIT','ELEV_BAROM',
                    'ELEV_BAROM_UNIT','ELEV_AIR','ELEV_AIR_UNIT','ELEV_ZERODAT','ELEV_ZERODAT_UNIT','ELEV_UNK','ELEV_UNK_UNIT','LAT_DEC',
                    'LON_DEC','LAT_LON_PRECISION','RELOCATION','UTC_OFFSET','OBS_ENV','PLATFORM','GHCNMLT_ID','COUNTY_FIPS_CODE',
                    'DATUM_HORIZONTAL','DATUM_VERTICAL','LAT_LON_SOURCE','IGRA_ID','HPD_ID']

    data = []  # Initialize empty list to store parsed data
    with open(filename, 'r') as file:  # Open file
        for line in file:  # Read file line by line
            row_data = parse_line(line, fmt, column_names)  # Parse each line
            data.append(row_data)  # Append parsed data to the list
            
    dfr = pd.DataFrame(data)  # Convert list of dicts to DataFrame
    
    # Subset the DataFrame based on the station ids in 'stnid'
    dfrs = dfr[dfr['GHCND_ID'].isin(stnid)].reset_index(drop=True)
    # Remove duplicate rows based on 'GHCND_ID' and keep the last occurrence
    dfrss = dfrs.drop_duplicates(subset=['GHCND_ID'], keep='last', inplace=False).reset_index(drop=True)

    return dfrss  # Return

    
def extract_elevation_srtm90(filepath):
    """
    This function extracts elevation data from an SRTM90 (Shuttle Radar Topography Mission) 
    netCDF file, handles duplicate latitude values and reorders the latitudes in descending order.

    Parameters:
    filepath (str): Path to the SRTM90 netCDF file.

    Returns:
    elev_srtm (DataArray): The cleaned elevation data.
    """

    # Open the netCDF file as an xarray Dataset
    ds = xr.open_dataset(filepath)

    # Extract the 'elevation' variable from the Dataset
    elev_srtm = ds['elevation']

    # Convert the 'lat' coordinate to a pandas Series
    lat_series = pd.Series(elev_srtm['lat'].values)

    # Identify duplicate latitude values in the Series
    duplicate_lats = lat_series.duplicated()

    # Invert the boolean values to get the unique latitude values
    unique_lats = ~duplicate_lats

    # Get the indices of the unique latitude values
    indices = np.where(~duplicate_lats)[0]

    # Subset the DataArray with the unique latitude values
    elev_srtm = elev_srtm[indices,:]

    # Reverse the order of 'lat' coordinates to descending
    elev_srtm = elev_srtm.sel(lat=elev_srtm['lat'][::-1])
    
    return elev_srtm  # Return the cleaned DataArray


def minmaxlatlon(lat, lon, sec):
    """
    This function calculates the minimum and maximum latitude and longitude within a given 
    number of seconds from the provided latitude and longitude.

    Parameters:
    lat (float): Latitude in decimal degrees.
    lon (float): Longitude in decimal degrees.
    sec (float): Number of seconds to add and subtract from the provided latitude and longitude.

    Returns:
    min_lat, max_lat, min_lon, max_lon (tuple): The minimum and maximum latitude and longitude values.
    """

    def decimal_to_dms(degrees):
        """
        This helper function converts decimal degrees to degrees, minutes, and seconds (DMS).
        
        Parameters:
        degrees (float): The decimal degrees to convert.

        Returns:
        d, m, s (tuple): The degrees, minutes, and seconds equivalent of the input decimal degrees.
        """
        d = int(degrees)  # Extract degrees
        m = int((degrees - d) * 60)  # Extract minutes
        s = (degrees - d - m/60) * 3600  # Extract seconds
        return d, m, s

    def dms_to_decimal(d, m, s):
        """
        This helper function converts degrees, minutes, and seconds (DMS) to decimal degrees.
        
        Parameters:
        d, m, s (tuple): The degrees, minutes, and seconds to convert.

        Returns:
        degrees (float): The decimal degrees equivalent of the input DMS.
        """
        return d + m/60 + s/3600  # Convert DMS to decimal degrees

    def add_seconds(d, m, s, seconds):
        """
        This helper function adds a given number of seconds to a given DMS.
        
        Parameters:
        d, m, s (tuple): The original DMS.
        seconds (float): The number of seconds to add.

        Returns:
        d, m, s (tuple): The new DMS after adding the input seconds.
        """
        total_seconds = d * 3600 + m * 60 + s + seconds  # Total seconds
        return decimal_to_dms(total_seconds / 3600)  # Convert total seconds back to DMS

    # Convert input lat/lon from decimal degrees to DMS
    lat_d, lat_m, lat_s = decimal_to_dms(lat)
    lon_d, lon_m, lon_s = decimal_to_dms(lon)

    # Calculate the minimum and maximum lat/lon by adding/subtracting the input seconds
    min_lat_d, min_lat_m, min_lat_s = add_seconds(lat_d, lat_m, lat_s, -sec)
    max_lat_d, max_lat_m, max_lat_s = add_seconds(lat_d, lat_m, lat_s, sec)
    min_lon_d, min_lon_m, min_lon_s = add_seconds(lon_d, lon_m, lon_s, -sec)
    max_lon_d, max_lon_m, max_lon_s = add_seconds(lon_d, lon_m, lon_s, sec)

    # Convert the min/max lat/lon from DMS back to decimal degrees
    min_lat = dms_to_decimal(min_lat_d, min_lat_m, min_lat_s)
    max_lat = dms_to_decimal(max_lat_d, max_lat_m, max_lat_s)
    min_lon = dms_to_decimal(min_lon_d, min_lon_m, min_lon_s)
    max_lon = dms_to_decimal(max_lon_d, max_lon_m, max_lon_s)

    return min_lat, max_lat, min_lon, max_lon  # Return the min

### Load station metadata 

*Only the station ids are read from this file. The latitude, longitude, and elevation information for these stations are read from EMSHR data. 
This ensures some of the inconsistencies such as typographical errors reported in the indiviual station metafile is auto corrected.*

<span style="color:red">Enter the path of the station metadata</span>


In [3]:
metaFile = 'stations_subset.txt'

In [4]:
df = pd.read_csv(metaFile, delimiter= '\t')
stnid = df['sid'].values

print(f'Number of stations to check for the elevation accuracy: {len(stnid)}')

Number of stations to check for the elevation accuracy: 12060


### Load EMSHR metadata

*The Enhanced Master Station History Report (EMSHR) is a compiled list of basic, historical information for every station in the station history database, beginning in the 1700s through the present.*

The file is downloaded here:

https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00771

*At the time of writing the code the file version is 202302*

In [5]:
filename = 'mshr_enhanced_202302.txt'

## read and subset EMSHR metadata
df_mshr = read_parse_and_subset_mshr(filename, stnid)
lat = df_mshr['LAT_DEC']
lon = df_mshr['LON_DEC']
elev_mshr = df_mshr['ELEV_GROUND']
ghcn_id = df_mshr['GHCND_ID']


### Replace '' with np.nan and convert the unit from feet to meters
total_iterations = len(elev_mshr)

for ii in range(total_iterations):
    elevr = elev_mshr.iloc[ii]
    if elevr == '':
        elev_mshr.iloc[ii] = np.nan
    else:
        elev_mshr.iloc[ii] = float(elevr) * 0.3048 # feet to meters
    
    progress = (ii + 1) / total_iterations * 100
    print(f"Processing: {progress:.1f}% complete", end="\r")

### Load SRTM 90m Elevation [Reference Elevation]

Look at ```saveSRTMelevationToNC.ipynb``` on how to generate the ```elevation_SRTM90m.nc```

The following code takes more than three minutes to execute. Please be patient.

In [6]:
srtmElevfile = 'elevation_SRTM90m.nc'

elev_srtm = extract_elevation_srtm90(srtmElevfile)

total_iterations = len(lat)

## Extract lattiude and longitude information for the stations.
elev_stn_srtm = []

for i, (latt, lonn) in enumerate(zip(lat, lon), 1):
    elev_stn_srtm.append(elev_srtm.sel(lat=latt, lon=lonn, method='nearest').values)
    
    progress = (i / total_iterations) * 100
    print(f"Processing: {progress:.1f}% complete", end="\r")

### Absolute difference between station elevation and SRTM-90m elevation

In [7]:
elev_diff = elev_mshr - elev_stn_srtm #elev_dem
ex = pd.Series(elev_diff)
# Take the absolute value of the elevation differences
abs_ex = ex.abs()

### Count the number of stations that fall within diferent absolute difference elevation ranges

In [8]:
# Bin the data into intervals of 10
bin_edges = list(range(0, int(abs_ex.max()) + 75, 75))
bins = pd.cut(abs_ex, bins=bin_edges)

# Count the occurrences of each bin
bin_counts = bins.value_counts().sort_index()

# Filter the bin_counts series to remove rows with count of 0
bin_counts_notzero = bin_counts[bin_counts != 0]

for idx, count in bin_counts_notzero.items():
    print(f"{idx}: {count}")

(0, 75]: 11549
(75, 150]: 243
(150, 225]: 56
(225, 300]: 27
(300, 375]: 16
(375, 450]: 3
(450, 525]: 5
(525, 600]: 2
(600, 675]: 2
(825, 900]: 2


### Threshold Test

<span style="color:red">Enter the threshold for the absolute elevation difference between station elevation and reference elevation</span>

In [9]:
threshold = 150 # in meters

In [10]:
index = np.where(abs_ex > threshold)[0]
print(f"Number of stations that have absolute elevation differences with reference elevation greater than 150 meters: {len(index)}")

Number of stations that have absolute elevation differences with reference elevation greater than 150 meters: 122


### One Minute precision box test
Test is done on stations that have absolute elevation differences greater than 150 meters  
*Find the minimum and maximum SRTM 90m elevation within one minute precision box centered at each station latitude and longitude.*  
*Check wheteher the station elevation falls between these values.*

<span style="color:red">Enter the half-width in seconds for the box centered around the station latitude and longitude</span>

In [11]:
sec = 30 # seconds, half width of the box. Full width is 1-minute

In [12]:
indexx = []
minlat = []
maxlat = []
minlon = []
maxlon = []


total_iterations = len(index)
for i, indd in enumerate(index):
    min_lat, max_lat, min_lon, max_lon = minmaxlatlon(float(lat[indd]), float(lon[indd]), sec)
    min_elev = elev_srtm.sel(lat=slice(min_lat, max_lat), lon=slice(min_lon, max_lon)).min(skipna=True)
    max_elev = elev_srtm.sel(lat=slice(min_lat, max_lat), lon=slice(min_lon, max_lon)).max(skipna=True)
    elevstn = elev_mshr[indd]
    if min_elev <= elevstn <= max_elev:
        pass
    else:
        minlat.append(min_lat)
        maxlat.append(max_lat)
        minlon.append(min_lon)
        maxlon.append(max_lon)
        indexx.append(indd)
    
    progress = (i + 1) / total_iterations * 100
    print(f"Processing: {progress:.1f}% complete", end="\r")

print(f"Number of stations that do not have elevation between the minimum and maximum of reference elevation in the one minute precision box: {len(indexx)}")

Number of stations that do not have elevation between the minimum and maximum of reference elevation in the one minute precision box: 79


### Latitude and longitude precisions

In [13]:
latlonpr = df_mshr['LAT_LON_PRECISION'].iloc[indexx]
precision = latlonpr.unique()
for pre in precision:
    if pre == '':
        print('Unknown')
    else:
        print(pre)

kis1= latlonpr[latlonpr == ''].index
kis2 = latlonpr[latlonpr == 'DDMM'].index
kis3 = latlonpr[latlonpr == 'DDMMSS'].index
kis4 = latlonpr[latlonpr == 'DDddddd'].index

Unknown
DDMM
DDMMSS
DDddddd


### Generate a map with stations that does not pass the threshold and one-minute precision box tests

The stations are grouped based on the latitude and longitude precision.  
*Please select the layer feature at the top-right corner of the map.*

In [14]:
# Create a map centered at the mean latitude and longitude of the dataframe

meanlat = np.mean(pd.to_numeric(lat, errors='coerce'))
meanlon = np.mean(pd.to_numeric(lon, errors='coerce'))

attribution = 'Map data: &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors,<a href="http://viewfinderpanoramas.org">SRTM</a> | Map style: &copy; <a href="https://opentopomap.org">OpenTopoMap</a> (<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)'

m = folium.Map(location=[meanlat, meanlon], zoom_start=6,  tiles='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png', attr=attribution)


# create feature groups for each layer
layer1 = folium.FeatureGroup(name='Unknown')
layer2 = folium.FeatureGroup(name='DDMM')
layer3 = folium.FeatureGroup(name='DDMMSS')
layer4 = folium.FeatureGroup(name='DDddddd')

# Add the elevation plugin to the map

# Add a marker for each location with a customized popup
for ii, indx in enumerate(kis1):
    popup_text = f"Station ID: {ghcn_id[indx]}<br>Lat: {lat[indx]}<br>Lon: {lon[indx]}<br>Elevation: {elev_mshr[indx]}<br>Ref Elevation: {elev_stn_srtm[indx]}<br>LATLON Precision: {df_mshr['LAT_LON_PRECISION'].iloc[indx]}"
    layer1.add_child(folium.Marker(location=[float(lat[indx]), float(lon[indx])], popup=folium.Popup(popup_text, max_width=300)))
    # Add a transparent rectangle to the map
    layer1.add_child(folium.Rectangle(bounds=[[minlat[indexx.index(indx)], minlon[indexx.index(indx)]], [maxlat[indexx.index(indx)], maxlon[indexx.index(indx)]]], fill=True, fill_color='#000000',
        fill_opacity=0.3,
        color=None,
        weight=2,
    ))

for ii, indx in enumerate(kis2):
    popup_text = f"Station ID: {ghcn_id[indx]}<br>Lat: {lat[indx]}<br>Lon: {lon[indx]}<br>Elevation: {elev_mshr[indx]}<br>Ref Elevation: {elev_stn_srtm[indx]}<br>LATLON Precision: {df_mshr['LAT_LON_PRECISION'].iloc[indx]}"
    layer2.add_child(folium.Marker(location=[float(lat[indx]), float(lon[indx])], popup=folium.Popup(popup_text, max_width=300)))
    # Add a transparent rectangle to the map
    layer2.add_child(folium.Rectangle(bounds=[[minlat[indexx.index(indx)], minlon[indexx.index(indx)]], [maxlat[indexx.index(indx)], maxlon[indexx.index(indx)]]], fill=True, fill_color='#000000',
        fill_opacity=0.3,
        color=None,
        weight=2,
    ))
    
for ii, indx in enumerate(kis3):
    popup_text = f"Station ID: {ghcn_id[indx]}<br>Lat: {lat[indx]}<br>Lon: {lon[indx]}<br>Elevation: {elev_mshr[indx]}<br>Ref Elevation: {elev_stn_srtm[indx]}<br>LATLON Precision: {df_mshr['LAT_LON_PRECISION'].iloc[indx]}"
    layer3.add_child(folium.Marker(location=[float(lat[indx]), float(lon[indx])], popup=folium.Popup(popup_text, max_width=300)))
    # Add a transparent rectangle to the map
    layer3.add_child(folium.Rectangle(bounds=[[minlat[indexx.index(indx)], minlon[indexx.index(indx)]], [maxlat[indexx.index(indx)], maxlon[indexx.index(indx)]]], fill=True, fill_color='#000000',
        fill_opacity=0.3,
        color=None,
        weight=2,
    ))


for ii, indx in enumerate(kis4):
    popup_text = f"Station ID: {ghcn_id[indx]}<br>Lat: {lat[indx]}<br>Lon: {lon[indx]}<br>Elevation: {elev_mshr[indx]}<br>Ref Elevation: {elev_stn_srtm[indx]}<br>LATLON Precision: {df_mshr['LAT_LON_PRECISION'].iloc[indx]}"
    layer4.add_child(folium.Marker(location=[float(lat[indx]), float(lon[indx])], popup=folium.Popup(popup_text, max_width=300)))
    # Add a transparent rectangle to the map
    layer4.add_child(folium.Rectangle(bounds=[[minlat[indexx.index(indx)], minlon[indexx.index(indx)]], [maxlat[indexx.index(indx)], maxlon[indexx.index(indx)]]], fill=True, fill_color='#000000',
        fill_opacity=0.3,
        color=None,
        weight=2,
    ))
    
# add each layer to the map
m.add_child(layer1)
m.add_child(layer2)
m.add_child(layer3)
m.add_child(layer4)

# add layer control to the map
folium.LayerControl().add_to(m)
    
# Display the map
m

### Save information for the stations that failed two tests to a csv file.

In [15]:
ref_elev = pd.Series(np.array(elev_stn_srtm)[indexx])
ref_elev.index = indexx

ref_elev = pd.Series(np.array(elev_stn_srtm)[indexx])
ref_elev.index = indexx
ref_elev.name = 'REFERENCE ELEVATION'

elev_absdiff = abs_ex[indexx]
elev_absdiff.name = 'ABSOLUTE DIFFERENCE'

cols = ['GHCND_ID', 'LAT_DEC', 'LON_DEC', 'LAT_LON_PRECISION', 'ELEV_GROUND']
df_check = df_mshr.loc[indexx, cols]
df_check = df_check.assign(**{ref_elev.name: ref_elev, elev_absdiff.name: elev_absdiff })

df_check.to_csv('failStations.csv',index=False)